In [ ]:
import requests
import pandas as pd
from google.colab import files

def get_df(name):
  url = f'https://raw.githubusercontent.com/iGEM-UGM/iGEM-UGM-drylab/main/{name}'
  response = requests.get(url)
  with open(name, 'wb') as f:
      f.write(response.content)

  return(pd.read_csv(name))

def save_data(df, filepath):
  df.to_csv(filepath, index=False)
  # Download the file
  files.download(filepath)

#1.0-kqs-gathering-sequence

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium

# Install xvfb
apt install -y xvfb

# Install Selenium-Profiles
pip uninstall -y selenium_profiles
pip install --no-cache-dir selenium_profiles>=2.2.6

# pip install https://github.com/kaliiiiiiiiii/Selenium-Profiles/archive/refs/heads/dev.zip # dev-branch

# install python packages
pip install google-colab-shell
pip install webdriver-manager
pip install Pyvirtualdisplay

In [ ]:
import pandas as pd
import time
def scrap_data_design(link, trial):
  driver.get(f'{link}/{trial}')  # test client hints
  # Add a time delay to allow for page loading
  time.sleep(2)  # Adjust the delay as needed
  # dropdown sequences
  button = driver.find_elements(By.CSS_SELECTOR, "div.title")[1]
  button.click()
  showscreen(driver)

  #scrap the table of result
  tables = driver.find_elements(By.CLASS_NAME, "ui.table")
  table_data = []
  for tab in tables:
    rows = tab.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text for cell in cells]
        table_data.append(row_data)

  df = pd.DataFrame(table_data, columns=["Domain_Strand", "Sequence"])
  return(df)

In [ ]:
def get_design_data(link):
  driver.get(link+'/0')
  # Find the menu element by its class name
  menu = driver.find_element(By.CLASS_NAME, "dropdown.icon")
  # Click the menu element
  menu.click()
  menu = driver.find_element(By.CSS_SELECTOR, "div.menu.transition.visible")
  # Find the items within the menu
  items = menu.find_elements(By.CLASS_NAME, "item")

  # Get the count of items
  item_count = len(items)
  print("Number of items:", item_count)

  # Create an empty DataFrame
  datas = pd.DataFrame()

  # Iterate over the items
  for i in range(item_count):
      # Scraping data using the scrap_data_design function
      data = scrap_data_design(link, i)

      n = len(data)  # The number of times to repeat the value
      result = [i+1] * n
      data['Trial'] = result
      # Append data to the datas DataFrame
      datas = datas.append(data, ignore_index=True)
  return(datas)

In [ ]:
driver = mydriver.start()
# Login
driver.get('https://www.nupack.org')

# Find the button by its CSS selector
button = driver.find_element(By.CSS_SELECTOR, "a.item[href='/auth/log-in']")

# Click the button
button.click()

# Find the input field by its class name
input_field = driver.find_element(By.CLASS_NAME, "ui input")

# Clear any existing text in the input field (optional)
input_field.clear()

# Enter the desired email address
email_address = mail #add your email here
input_field.send_keys(email_address)
input_element = driver.find_element(By.CLASS_NAME, "ui.fluid.icon.input")

# Find the input field within the parent element
password_field = input_element.find_element(By.TAG_NAME, "input")
password_field.send_keys(password) #add your password here

eye_icon = driver.find_element(By.CSS_SELECTOR, ".eye.fitted.link.icon")
eye_icon.click()
# Find the button
button = driver.find_element(By.CLASS_NAME, "ui.button.orange")
button.click()

# start scrapping
link = 'https://www.nupack.org/design/results/a10996f7-3a7c-49eb-ba3c-bf682effc883'

output_sequence = get_design_data(link)

# Close the browser
driver.quit()

# save link into csv data downloaded automatically
code = link.split('/')[-1]
name = f"table_data_{code}.csv"
save_data(output_sequence, name)

NameError: ignored

# 2.0-kqs-parameter-gathering-nupack-analysis

In [ ]:
import nupack
from nupack import *
def get_rbs_index(c):
    a = ''
    for i in c.strands:
        a += str(i)
    rbs = "UAGAGGAGAUG"
    index = a.find(rbs)
    return(int(index))

def calculate_gc_percentage(string):
    gc_count = string.count("G") + string.count("C")
    total_count = len(string)
    gc_percentage = (gc_count / total_count) * 100

    return(gc_percentage)

import re
import numpy as np
def check_illegal_sites(sequence, enzymes):
    illegal_sites = 0
    for enzyme, site in enzymes.items():
        matches = re.finditer(site, sequence)
        for match in matches:
            illegal_sites += 1

    return(illegal_sites)

def get_index(text):
    periods = []
    parentheses = []

    for match in re.finditer(r'\.|[()]', text):
        if match.group() == '.':
            periods.append(match.start())
        elif match.group() == '(' or match.group() == ')':
            parentheses.append(match.start())
    return periods, parentheses

def get_prob(target, matrix):
    # target = str(tube_results.complexes[Complex([b, a, c], name='(b+a+c)')].mfe[0][0]).replace("+", "")
    target = target.replace('+', '')
    # matrix = pairs(Complex([b, a, c], name='(b+a+c)'), model=model1).to_array()
    diagonal_elements = np.diag(matrix)
    non_diagonal_array = matrix - diagonal_elements
    prob = []
    for i in range(len(target)):
        if target[i] == ')' or target[i] == '(':
            prob.append(np.max(non_diagonal_array[i]))
        elif target[i] == '.':
            prob.append(diagonal_elements[i])
        else:
            continue
    return prob

def get_probability_onoff(index, data):
    prob = []
    for i in index:
        prob.append(data[i])
    return prob

def on_off(text, data):
    periods, parentheses = get_index(text)
    prob_on  = []
    prob_off = []
    prob_on = get_probability_onoff(periods, data)
    prob_off = get_probability_onoff(parentheses, data)
    return(prob_on, prob_off)

def on_off_level(text, data):
    prob_on, prob_off = on_off(text, data)
    on_level = sum(prob_on)/len(text)
    off_level = sum(prob_off)/len(text)
    return(on_level, off_level)

def stats(lists):
    try:
        mean = sum(lists) / len(lists)
        squared_differences = [(x - mean)**2 for x in lists]
        variance = sum(squared_differences) / len(lists)
        standard_deviation = variance**0.5
        sum_res = sum(lists)

        return(mean, sum_res, standard_deviation)
    except ZeroDivisionError:
        print("Error: Division by zero.")
        return 0, 0, 0

def get_analyze(t, df):
    seq = str.upper((df.loc[(df['Trial'] == t) & (df['Domain_Strand'] == 'LiRA'), 'Sequence'].values)[0])
    mir21 = 'UAGCUUAUCAGACUGAUGUUGA'
    mir92a = 'UAUUGCACUUGUCCCGGCCUGU'


    # define illegal sites enzymes
    '''
    Ecori: GAATTC GAAUUC
    Xbai: TCTAGA UCUAGA
    Spei: ACTAGT ACUAGU
    Psti: CTGCAG CUGCAG
    noti:GCGGCCGC
    '''
    enzymes = {
      "EcoRI": "GAATTC",
      "EcoRI2": "GAAUUC",
      "XbaI": "TCTAGA",
      "XbaI2": "UCUAGA",
      "SpeI": "ACTAGT",
      "SpeI2": "ACUAGU",
      "PstI": "CTGCAG",
      "PstI2": "CUGCAG",
      "NotI": "GCGGCCGC"
    }
    # analysis job

    # specify strands
    a = Strand(seq, name='a')
    b = Strand(mir21, name='b')
    c = Strand(mir92a, name='c')

    # specify tubes
    t1 = Tube(strands={a: 1e-6, b: 1e-6, c: 1e-6}, complexes=SetSpec(max_size=3, include=[[a,b], [a,c], [a,b,c]]), name='t1')
    # t2 = Tube(strands={a: 1e-10, b: 1e-9}, complexes=SetSpec(max_size=2), name='t2')

    # analyze tubes
    model1 = Model()
    tube_results = tube_analysis(tubes=[t1], model=model1, compute=['mfe','ensemble_size'])

    # Split the ASCII result into lines
    lines = str(tube_results).split('\n')

    # Extract the column names
    columns = lines[0].split()

    # Create an empty list to store the data rows
    concen_data = []

    # Iterate over the lines starting from the second line
    for line in lines[1:]:
        # Split the line into individual values
        values = line.split()
        # Append the values as a row to the data list
        concen_data.append(values)

    # Iterate through the list and find the index containing 'Concentration'
    index = None
    for i, sublist in enumerate(concen_data):
        if 'Concentration' in sublist:
            index = i
            break

    new_data = concen_data[index+1:]
    # Create the dataframe
    concen = pd.DataFrame(new_data[1:], columns=new_data[0][:-1], )

    #  data get from each sequence
    c0 = Complex([a], name='(a)')
    c1 = Complex([a, c, b], name='(a+c+b)')
    c2 = Complex([a, b, c], name='(a+b+c)')
    c3 = Complex([a, b], name='(a+b)')
    c4 = Complex([a, c], name='(a+c)')
    c5 = Complex([a, a], name='(a+a)')

    data = dict ()
    data['sequence'] = seq
    for c in [c0, c1, c2, c3, c4, c5]:
        data[str(c.name) + '_mfe'] =  tube_results.complexes[c].mfe_stack
        data[str(c.name) + '_target']  = str(tube_results.complexes[c].mfe[0][0])
        data[str(c.name) + '_pfunc'] =  tube_results.complexes[c].pfunc
        data[str(c.name) + '_free_energy'] =  tube_results.complexes[c].free_energy
        data[str(c.name) + '_ensemble_size'] =  tube_results.complexes[c].ensemble_size
        data[str(c.name) + '_mfe_energy'] =  tube_results.complexes[c].mfe_stack
        data[str(c.name) + '_mfe_stack_energy'] =  tube_results.complexes[c].mfe[0][2]
        data[str(c.name) + '_t1'] = concen.loc[concen['Complex'] == str(c.name), 't1'].values[0]
        data[str(c.name) + '_t1_rank'] = concen.index[concen['Complex'] == str(c.name)][0]
        data['on_' + str(c.name)] = len(re.findall(r'\.', str(tube_results.complexes[c].mfe[0][0]))[get_rbs_index(c):get_rbs_index(c)+18])
        print(get_rbs_index(c))
        data['off_'+ str(c.name)] = len(re.findall(r'[()]', str(tube_results.complexes[c].mfe[0][0]))[get_rbs_index(c):get_rbs_index(c)+18])
        data['prob_matrix_'+ str(c.name)] = pairs(c, model=model1).to_array()

        data['prob_' + str(c.name)] = get_prob(str(tube_results.complexes[c].mfe[0][0]).replace("+", ""), pairs(c, model=model1).to_array())
        data['prob_on_' + str(c.name)], data['prob_off_' + str(c.name)] = on_off(str(tube_results.complexes[c].mfe[0][0])[get_rbs_index(c):get_rbs_index(c)+18], data['prob_' + str(c.name)])
        data['on_level_' + str(c.name)], data['off_level_' + str(c.name)] = on_off_level(str(tube_results.complexes[c].mfe[0][0])[get_rbs_index(c):get_rbs_index(c)+18], data['prob_' + str(c.name)])
        data['on_mean_' + str(c.name)], data['on_sum_' + str(c.name)], data['on_stdev_' + str(c.name)] = stats(data['prob_on_' + str(c.name)])
        data['off_mean_' + str(c.name)], data['off_sum_' + str(c.name)], data['off_stdev_' + str(c.name)] = stats(data['prob_off_' + str(c.name)])

    data['defect'] = defect(strands=[seq], structure=data['(a)_target'], model=model1)
    data['structure_prob'] = structure_probability(strands=[seq], structure=data['(a)_target'], model=model1)
    # data['prob_matrix'] = pairs(strands=[seq], model=model1).to_array()
    # data['prob_diagonal'] = np.diagonal(pairs(strands=[seq], model=model1).to_array())
    data['gc_content'] = calculate_gc_percentage(seq)
    data['illegal_count'] = check_illegal_sites(seq, enzymes)
    return(data)

In [ ]:
import pandas as pd

df = pd.read_csv('./table_data_a10996f7-3a7c-49eb-ba3c-bf682effc883.csv')
datas = []
for i in df['Trial'].unique().tolist():
    datas.append(get_analyze(i, df))
result = pd.DataFrame(datas)
result.head(20)
result.info()

In [ ]:
def get_on_off_each_seq(cols_complex, result)
  for i in cols_complex:
      print(i)
      on_off_ratio = []
      on_off_minus = []
      for j in range(len(result)):
          # print(result['on_level_' + str(i)][j])
          # print(result['off_level_' + str(i)][j])
          ratio = result['on_level_' + str(i)][j] / result['off_level_' + str(i)][j]
          minus = result['on_level_' + str(i)][j] - result['off_level_' + str(i)][j]
          # print(ratio)
          on_off_ratio.append(ratio)
          on_off_minus.append(minus)
  return(on_off_ratio, on_off_minus)

In [ ]:
def get_metrics(cols_complex, result):
  ratio_avg_list = []
  minus_avg_list = []
  on_level_avg_list = []
  off_level_avg_list = []
  w_ratio_avg_list = []
  w_minus_avg_list = []
  w_on_level_avg_list = []
  w_off_level_avg_list = []
  for j in range(len(result)):
      ratio_avg = 0
      minus_avg = 0
      on_level_avg = 0
      off_level_avg = 0
      weighted_on_level_avg = 0
      weighted_off_level_avg = 0
      weighted_minus_avg = 0
      weighted_ratio_avg = 0
      for i in cols_complex:
          # complex regression: ['(a+c+b)', '(a+b+c)', '(a+b)', '(a+c)']
          ratio_avg += result['on_off_ratio_' + str(i)][j]
          minus_avg += result['on_off_minus_' + str(i)][j]
          on_level_avg += result['on_level_' + str(i)][j]
          off_level_avg += result['off_level_' + str(i)][j]
          weighted_on_level_avg += (1/(result[str(i) + '_t1_rank'][j]+1))*result['on_level_' + str(i)][j]
          weighted_off_level_avg += (1/(result[str(i) + '_t1_rank'][j]+1))*result['off_level_' + str(i)][j]
          weighted_ratio_avg += (1/(result[str(i) + '_t1_rank'][j]+1))*result['on_off_ratio_' + str(i)][j]
          weighted_minus_avg += (1/(result[str(i) + '_t1_rank'][j]+1))*result['on_off_minus_' + str(i)][j]
          # print(result['on_off_ratio_' + str(i)][j])
      ratio_avg_list.append(ratio_avg/len(cols_complex))
      minus_avg_list.append(ratio_avg/len(cols_complex))
      on_level_avg_list.append(on_level_avg/len(cols_complex))
      off_level_avg_list.append(off_level_avg/len(cols_complex))
      #weighted
      w_minus_avg_list.append(weighted_ratio_avg/len(cols_complex))
      w_ratio_avg_list.append(weighted_ratio_avg/len(cols_complex))
      w_on_level_avg_list.append(weighted_on_level_avg/len(cols_complex))
      w_off_level_avg_list.append(weighted_off_level_avg/len(cols_complex))
      return(ratio_avg_list, minus_avg_list, on_level_avg_list, off_level_avg_list,
             w_ratio_avg_list, w_minus_avg_list, w_on_level_avg_list, w_off_level_avg_list)

In [ ]:
def get_complex(df):
  cols_complex = []
  for i in df.columns[df.columns.str.startswith('on_level_')].tolist()[1:-1]:
    cols_complex.append(i.split('on_level_')[1])
  print(cols_complex)
  return(cols_complex)

In [ ]:
cols_complex = get_complex(result)
result['on_off_ratio_' + str(i)], result['on_off_minus_' + str(i)] = get_on_off_each_seq(cols_complex, result)
result['on_off_ratio_avg'], result['on_off_minus_avg'], result['on_level_avg'], result['off_level_avg'], result['weighted_on_off_ratio_avg'], result['weighted_on_off_minus_avg'], result['weighted_on_level_avg'], result['weighted_off_level_avg'] = get_metrics(cols_complex, result)
save_data(result, 'nupack_params.csv')

# 2.1-kqs-parameter-gathering-mfe

In [ ]:
!pip install ViennaRNA

In [ ]:
def mfe_segment(data):
  list_domain = ['ABS',
                  'ABS_',
                  'ATS',
                  'Complementstart',
                  'DBSCom',
                  'Loop',
                  'RBS',
                  'StartCodon',
                  '_DBS',
                  'loopCom',
                  'loopRBS',
                  'miR21bottomseal1',
                  'miR21bottomseal2',
                  'miR21topseal',
                  'miR92bottomseal1',
                  'miR92bottomseal2',
                  'miR92topseal',
                  'toeholdmiR92',
                  'toeholdmir21']
  list_segment = [['ABS', 'Complementstart', 'ABS_', 'Loop', 'ATS', 'loopRBS', 'miR21bottomseal1'],
                  ['toeholdmir21', 'miR21topseal', 'miR21bottomseal2', 'miR92bottomseal1', 'toeholdmiR92', 'miR92topseal'],
                  ['miR92bottomseal2', 'RBS', 'loopCom', '_DBS', 'StartCodon', 'DBSCom']]
  res = dict()
  list_name = ['ABS-miR21bottomseal1', 'toeholdmir21-miR92topseal', 'miR92bottomseal2-end']
  for j, name in zip(list_segment, list_name):
    # get The RNA sequence segment
    seq = ''
    for i in j:
      # print(i)
      dom = data.loc[data['Domain_Strand'] == str(i), 'Sequence'].values[0]
      seq += dom
    seq = seq.lower()
    print(seq)
    # compute minimum free energy (MFE) and corresponding structure
    (ss, mfe) = RNA.fold(seq)
    res['mfe_' + name] = mfe
  return(res)

In [ ]:
import RNA
import pandas as pd
url='https://drive.google.com/file/d/1ulrMr39Uy2RaSLEaGBGOVwtSGhN11gPS/view?usp=sharing'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id

df = pd.read_csv(dwn_url)
datas = []
for i in df['Trial'].unique().tolist():
    datas.append(mfe_segment(df[df['Trial']==i]))

#3.0-kqs-modelling

In [ ]:
df1 = get_df('nupack_10_weighted.csv')
df2 = get_df('mfe_data_10trial_revisi.csv')
df = pd.concat([df1, df2], axis=1)
df.head()

In [ ]:
# get columns that has variation of complexes
cols_complex = []
for i in df.columns[df.columns.str.startswith('on_level_')].tolist()[1:-1]:
  cols_complex.append(i.split('on_level_')[1])

In [ ]:
# get the list of columns to get into regression
  _param = ['_mfe',
          '_pfunc',
          '_free_energy',
          '_ensemble_size',
          '_mfe_energy',
          '_mfe_stack_energy',
          '_t1',
          '_t1_rank']
  param_  = ['on_level_',
            'off_level_',
            'on_mean_',
            'on_sum_',
            'on_stdev_',
            'off_mean_',
            'off_sum_',
            'off_stdev_']
  param = ['mfe_ABS-miR21bottomseal1', 'mfe_toeholdmir21-miR92topseal',
          'mfe_miR92bottomseal2-end',
          'gc_content', 'structure_prob', 'defect']
import numpy as np
def prepro(df, mode = 'infinity'):
  # change datatype to float
  df['(a)_pfunc'] = df['(a)_pfunc'].astype(float)
  df['(a)_ensemble_size'] = df['(a)_ensemble_size'].astype(float)
  df['(a+c+b)_ensemble_size'] = df['(a+c+b)_ensemble_size'].astype(float)
  df['(a+b+c)_ensemble_size'] = df['(a+b+c)_ensemble_size'].astype(float)
  df['(a+b)_ensemble_size'] = df['(a+b)_ensemble_size'].astype(float)
  df['(a+c)_ensemble_size'] = df['(a+c)_ensemble_size'].astype(float)
  df['(a+a)_ensemble_size'] = df['(a+a)_ensemble_size'].astype(float)
  df_non_inf = pd.DataFrame()
  if mode == 'infinity':
    # Check which rows contain infinity values
    rows_with_inf = df.isin([np.inf]).any(axis=1)

    # Drop rows with infinity values
    df_non_inf = df.drop(index = pd.Series(rows_with_inf)[pd.Series(rows_with_inf)].index[0])
  return(df, df_non_inf)

def get_param(df, _param, cols_complex):

  params_all = []
  for i in cols_complex:
    for j in _param:
      params_all.append(str(i+j))
    # for k in param_:
    #   params_all.append(str(k+i))
  params_all.extend(param)
  return(params_all)
print(len(cols_complex)*(len(_param))+len(param))
len(params_all)

In [ ]:
# modelling
def get_mse_rmse(y_test, y_pred):
  mse = mean_squared_error(y_test, y_pred)
  rmse = np.sqrt(mse)
  return mse, rmse

def linreg(X, y):
  sc_X = StandardScaler()
  X = sc_X.fit_transform(X)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Build linear regression model
  model = LinearRegression()
  model.fit(X_train, y_train)
  # Make predictions on testing set
  y_pred = model.predict(X_test)

  mse, rmse = get_mse_rmse(y_test, y_pred)

  print('MSE:', mse)
  print('RMSE:', rmse)
  return(model)

# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Split data into training and testing sets
X = df_non_inf[params_all]
y = df_non_inf['on_off_ratio_avg']
model = linreg(X, y)


In [ ]:
def display_metrics(sf, sc):
  display(pd.DataFrame({
      'Features': sf,
      'Coefficients': sc,
      'Absolute Coefficients': np.abs(sc)
  }))

def metrics(model, params_all):
  # Get the coefficients of the linear regression model
  coefficients = model.coef_

  # Sort the coefficients in descending order
  sorted_indices = np.argsort(coefficients)[::-1].tolist()
  print(sorted_indices)
  sorted_coefficients = coefficients[sorted_indices]
  sorted_features = sorted(params_all, key=lambda x: sorted_indices.index(params_all.index(x)))

  # Print or visualize the feature importances
  for feature, importance in zip(sorted_features, sorted_coefficients):
      print(f"{feature}: {importance}")

  # Create a bar plot of feature importances
  plt.bar(range(len(sorted_features)), sorted_coefficients)
  plt.xticks(range(len(sorted_features)), sorted_features, rotation='vertical')
  plt.xlabel('Features')
  plt.ylabel('Coefficient')
  plt.title('Feature Coefficients')
  plt.show()
  return(zip(sorted_features, sorted_coefficients))

#4.0-kqs-EDA